In [1]:
from functions import sample_hyperparameters, get_features, discretize_trajectories, get_feature_traj_paths, bootstrap_ts2
import pickle
import numpy as np
import pyemma as pm
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join

In [2]:
search_space = pickle.load(open('common/hp_search_space.p', 'rb'))
lags = (np.arange(1, 25)*10).astype(int)
features = ['alpha', 'contact', 'phi_psi']
sim_names = ['transition_state', 'reactant']

# Introduction

In this notebook we'll determine an appropriate lag time.
To do this, we'll bootstrap the slowest timescale and find where it converges. 
We'll do this for every transition state/reaction/feature combination with 5 random draws of the other hypeparameters. 
Convergence will be judged by the first change in T2 less than 100 ns over 1ns of lag time. The lag time will be the smallest lag time from among the sampled hyperparameters. 

First, we find an appropriate number of bootstrap samples. 

Then we do this for all other combinations of hyperparameters and simulation systems (reactant/transition state). 

In another workbook, we'll analyse the results. 

In [3]:
np.random.seed(93475)

traj_paths = get_feature_traj_paths(sim_names[0], features[0])
hps = sample_hyperparameters(search_space)
trajs = get_features(traj_paths)
bs_its = bootstrap_ts2(n=100, hps=hps, trajs=trajs, lags=lags)

KeyboardInterrupt: 

In [ ]:
mean_bs_its = np.mean(bs_its, axis=0)

cum_mean_its = np.cumsum(bs_its, axis=0)/(np.arange(bs_its.shape[0])[:, None, None]+1)

diffs = np.sqrt(np.mean((np.log10(cum_mean_its[:-1, :, 0]) - np.log10(cum_mean_its[1:, :, 0]))**2, axis=1))

with sns.plotting_context('talk', font_scale=1):
    plt.plot(diffs)
#     plt.yscale('log')
    plt.xlabel('bootstrap samples, i')
    plt.ylabel('RMSD $\log(t2_{i}) - \log(t2_{i+1})$')

40 bootstrap samples seems fine. 

In [ ]:
np.random.seed(93475)

hps_per_feature = 5
n_bootstraps = 40

for sim in sim_names:
    for feature in features:
        
        traj_paths = get_feature_traj_paths(sim, feature)
        trajs = get_features(traj_paths)

        for i in range(hps_per_feature):
            hps = sample_hyperparameters(search_space)
            fname = f"{sim}_{feature}_l{hps['vamp']['lag']}_d{hps['vamp']['dim']}_n{hps['cluster_kmeans']['k']}.p"
            print(fname)

            try:
                bs_its = bootstrap_ts2(n=n_bootstraps, hps=hps, trajs=trajs, lags=lags)
                pickle.dump(file=open(join('lag_determination', fname), 'wb'), obj=[hps, bs_its])
            except: 
                print('ERROR with ', fname)